In [0]:
#NLTK kümesinin yüklenme işlemi
!pip install nltk

     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 97 kB 10.0 MB/s 
     |████████████████████████████████| 764 kB 43.7 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
RANDOM_SEED = 42 # K-means kümelemeyi çalışmak için
VOCAB_SIZE = 1000 #  Kelime hazinesi olarak tutulacak kelime sayısı
MIN_DOC_FREQ = 10 # kelime dağarcığına dahil edilmesi için bir kelimenin görünmesi gereken minimum belge sayısı
N_GRAMS = 2 # n-gram sayısı (gerekirse)
N_FEATURES = 200 # varsayılan gömme vektör boyutu (HashingTF veya daha sonra Word2Vec kullanılıyorsa)

In [0]:
#Gerekli kütüphanelerin import edilmesi
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,NaiveBayes,RandomForestClassifier
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.sql.functions import udf, col, lower, trim, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer # BE SURE NLTK IS INSTALLED ON THE CLUSTER USING THE 
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF, NGram
from pyspark.ml import Pipeline

from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.mllib.evaluation import MulticlassMetrics

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt


In [0]:
#obje oluşturmak için sparksession.SparkSessionMevcut olanı alır veya mevcut değilse, bu oluşturucuda ayarlanan seçeneklere göre yeni bir tane oluşturur.
spark = SparkSession \
    .builder \
    .appName("Spark ML example on Twitter data ") \
    .getOrCreate()

In [0]:
# CSV dosyasını okuma.Eğitim veri setini okuma işlemi
#dataset = "/FileStore/tables/testdata_manual_2009_06_14-4.csv"
dataset = "dbfs:/datasets/train.csv"
twitter_df = spark.read.csv(dataset,header = 'False',inferSchema='True')

In [0]:
# Polarite değerlerinin sayılarını öğrenme.
twitter_df.groupBy("_c0").count().show()

+---+------+
|_c0| count|
+---+------+
|0.0|784103|
|4.0|778833|
+---+------+



In [0]:
# Eğer veri kümesinin hepsini kullanmayacaksak.
#twitter_df,df2 = twitter_df.randomSplit([0.001, 0.80],seed=1234)

In [0]:
#csv veri kümesini gösterme.
display(twitter_df)
print(twitter_df.count())

1562936


_c0,_c1,_c2,_c3,_c4,_c5
0.0,1.467810369E9,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0.0,1.467810672E9,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
0.0,1.467810917E9,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
0.0,1.467811184E9,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
0.0,1.467811193E9,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
0.0,1.467811372E9,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
0.0,1.467811592E9,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
0.0,1.467811594E9,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?"
0.0,1.467811795E9,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
0.0,1.467812025E9,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [0]:
#The data is a CSV with emoticons removed. Data file format has 6 fields:
#0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
#1 - the id of the tweet (2087)
#2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
#3 - the query (lyx). If there is no query, then this value is NO_QUERY.
#4 - the user that tweeted (robotickilldozr)
#5 - the text of the tweet (Lyx is cool)
#Bu kısımda sütunlara alias verildi.

twitter_df = twitter_df.select(col("_c0").alias("polarite"), col("_c1").alias("id"), col("_c2").alias("date"), col("_c3").alias("query"), col("_c4").alias("user_name"), col("_c5").alias("tweet"))
twitter_df.show()

+--------+-------------+--------------------+--------+---------------+--------------------+
|polarite|           id|                date|   query|      user_name|               tweet|
+--------+-------------+--------------------+--------+---------------+--------------------+
|     0.0|1.467810369E9|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0.0|1.467810672E9|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|     0.0|1.467810917E9|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0.0|1.467811184E9|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|     0.0|1.467811193E9|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|     0.0|1.467811372E9|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|     0.0|1.467811592E9|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|          Need a hug|
|     0.0|1.467811594E9|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish 

In [0]:
#Aliaslar oluşturulduktan sonra tekrar kolonları görüntülemek için 
twitter_df.printSchema()

root
 |-- polarite: double (nullable = true)
 |-- id: double (nullable = true)
 |-- date: string (nullable = true)
 |-- query: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- tweet: string (nullable = true)



In [0]:
#Eğitim veri setindeki veri sayısını görüntülemek için
count = twitter_df.count()
count

Out[13]: 1562936

In [0]:
#Veri kümesi açıklamasını veriyor.Bir veri çerçevesinin veya bir dizi sayısal değerin yüzdelik oranı, ortalama, standart vb. gibi bazı temel istatistiksel ayrıntılarını görüntülemek için kullanılır.

twitter_df.describe().show()

+-------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|summary|          polarite|                  id|                date|   query|           user_name|               tweet|
+-------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|  count|           1562936|             1562936|             1562936| 1562936|             1562936|             1562936|
|   mean|1.9932562817671358| 1.998668614539793E9|                null|    null|4.5363099048052435E9|                null|
| stddev|1.9999892703519166|1.9380902534165788E8|                null|    null|5.286480081449335...|                null|
|    min|               0.0|       1.467810369E9|Fri Apr 17 20:30:...|NO_QUERY|        000catnap000|!  . So not payin...|
|    max|               4.0|       2.329205794E9|Wed May 27 07:27:...|NO_QUERY|          zzzzeus111|�����ߧ�ǿ�����ж�؜�...|
+-------+---------------

In [0]:
#istediğimiz sütundaki verilerden kaçar tane tweet atılmış ise sayısına ulaşabiliriz.
twitter_df.groupBy("polarite").count().show()

+--------+------+
|polarite| count|
+--------+------+
|     0.0|784103|
|     4.0|778833|
+--------+------+



In [0]:
#Eğitim veri seti içerisindeki Null değerleri ve Null değer sayısını yazdırmak için oluşturulan fonksiyon
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [0]:
# Null değerlerin sayını gösmek için
null_columns_count_list = null_value_count(twitter_df)
null_columns_count_list

In [0]:
twitter_df.show()

+--------+-------------+--------------------+--------+---------------+--------------------+
|polarite|           id|                date|   query|      user_name|               tweet|
+--------+-------------+--------------------+--------+---------------+--------------------+
|     0.0|1.467810369E9|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0.0|1.467810672E9|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|     0.0|1.467810917E9|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0.0|1.467811184E9|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|     0.0|1.467811193E9|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|     0.0|1.467811372E9|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|     0.0|1.467811592E9|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|          Need a hug|
|     0.0|1.467811594E9|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish 

In [0]:
twitter_df = twitter_df.drop("user_name_tweet","polarite_index","user_name_index","tweet_index")
twitter_df.show()

+--------+-------------+--------------------+--------+---------------+--------------------+
|polarite|           id|                date|   query|      user_name|               tweet|
+--------+-------------+--------------------+--------+---------------+--------------------+
|     0.0|1.467810369E9|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0.0|1.467810672E9|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|     0.0|1.467810917E9|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0.0|1.467811184E9|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|     0.0|1.467811193E9|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|     0.0|1.467811372E9|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|     0.0|1.467811592E9|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|          Need a hug|
|     0.0|1.467811594E9|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish 

In [0]:
def clean_text(df, column_name="tweet",column_polarite="polarite"):
    """ 
    Bu işlev, ham metin verilerini alır ve aşağıdaki adımlardan oluşan standart bir NLP ön işleme      hattı uygular:
            - Text cleaning
      - Tokenization Giriş dizesini küçük harfe dönüştüren ve ardından beyaz boşluklarla bölen bir belirteç.
      - Stopwords removal Durdurma sözcüklerini girişten filtreleyen bir özellik dönüştürücü
      - Stemming (Snowball stemmer) Kelime kökünü bulmak için
    parameter: dataframe
    returns: the input dataframe along with the `cleaned_content` column as the results of the NLP preprocessing pipeline
    Doğal dil işleme (NLP), birçok veri bilimi sisteminde bir metni anlaması veya bu konuda mantık yürütmesi gereken önemli bir bileşendir
    """
    print("***** Text Preprocessing Pipeline *****\n")
    # 1. Metin temizleme
    print("# 1. Text Cleaning\n")
    # 1.a Case normalization
    print("1.a Case normalization:")
    lower_case_news_df = df.select(lower(col(column_name)).alias(column_name),(col(column_polarite)).alias(column_polarite))
    lower_case_news_df.show(10)
    # 1.b Trimming
    print("1.b Trimming:") # cümle başındaki ve sonunda boşlukları siler.
    trimmed_news_df = lower_case_news_df.select(trim(col(column_name)).alias(column_name),(col(column_polarite)).alias(column_polarite))
    trimmed_news_df.show(10)
    # 1.c Filter out punctuation symbols
    print("1.c Filter out punctuation:")# Noktalama işaretlerinin çıkarılması
    no_punct_news_df = trimmed_news_df.select((regexp_replace(col(column_name), "[^a-zA-Z\\s]", "")).alias(column_name),(col(column_polarite)).alias(column_polarite))
    no_punct_news_df.show(10)
    # 1.d Filter out any internal extra whitespace
    print("1.d Filter out extra whitespaces:") #Ekstra boşlukların filtrelenmesi
    cleaned_news_df = no_punct_news_df.select(trim(regexp_replace(col(column_name), " +", " ")).alias(column_name),(col(column_polarite)).alias(column_polarite))
    cleaned_news_df.show(10)
    # 2. Tokenization (split text into tokens)
    print("# 2. Tokenization:")#metni tokenlara bölün)
    tokenizer = Tokenizer(inputCol=column_name, outputCol="tokens")
    tokens_df = tokenizer.transform(cleaned_news_df).cache()
    tokens_df.show(10)
    # 3. Stopwords removal Stopworsleri kaldırma
    print("# 3. Stopwords removal:")
    stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="terms")
    terms_df = stopwords_remover.transform(tokens_df).cache()
    terms_df.show(10)
    # 4. Stemming (Snowball stemmer)
    print("# 4. Stemming:") #Kelime kökünü bulma
    stemmer = SnowballStemmer(language="english")
    stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
    terms_stemmed_df = terms_df.withColumn("terms_stemmed", stemmer_udf("terms")).cache()
    terms_stemmed_df.show(10)
    return terms_stemmed_df

In [0]:
#Fonksiyon ile temizlenen verileri yeni bir nesneye atıyoruz
clean_news_df = clean_text(twitter_df)

***** Text Preprocessing Pipeline *****

# 1. Text Cleaning

1.a Case normalization:
+--------------------+--------+
|               tweet|polarite|
+--------------------+--------+
|@switchfoot http:...|     0.0|
|is upset that he ...|     0.0|
|@kenichan i dived...|     0.0|
|my whole body fee...|     0.0|
|@nationwideclass ...|     0.0|
|@kwesidei not the...|     0.0|
|          need a hug|     0.0|
|@loltrish hey  lo...|     0.0|
|@tatiana_k nope t...|     0.0|
|@twittera que me ...|     0.0|
+--------------------+--------+
only showing top 10 rows

1.b Trimming:
+--------------------+--------+
|               tweet|polarite|
+--------------------+--------+
|@switchfoot http:...|     0.0|
|is upset that he ...|     0.0|
|@kenichan i dived...|     0.0|
|my whole body fee...|     0.0|
|@nationwideclass ...|     0.0|
|@kwesidei not the...|     0.0|
|          need a hug|     0.0|
|@loltrish hey  lo...|     0.0|
|@tatiana_k nope t...|     0.0|
|@twittera que me ...|     0.0|
+----------

In [0]:
def extract_tfidf_features(df, column_name="terms_stemmed"):
    """ 
    Bu işlev, metin verilerini alır ve onu bir terim frekansı-ters belge frekans vektörüne dönüştürür.
    parameter: dataframe
    returns: dataframe with tf-idf vectors
    """


    cv = CountVectorizer(inputCol=column_name, outputCol="tf_features", vocabSize=VOCAB_SIZE, minDF=MIN_DOC_FREQ)
    idf = IDF(inputCol="tf_features", outputCol="features")

    pipeline = Pipeline(stages=[cv, idf]) 
    features = pipeline.fit(df)
    tf_idf_features_df = features.transform(df).cache()

    return tf_idf_features_df


In [0]:
#TF-IDF ile Özellik Çıkarma
features = extract_tfidf_features(clean_news_df)

In [0]:
features.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               tweet|polarite|              tokens|               terms|       terms_stemmed|         tf_features|            features|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|switchfoot httptw...|     0.0|[switchfoot, http...|[switchfoot, http...|[switchfoot, http...|(1000,[3,12,49,29...|(1000,[3,12,49,29...|
|is upset that he ...|     0.0|[is, upset, that,...|[upset, cant, upd...|[upset, cant, upd...|(1000,[9,11,78,18...|(1000,[9,11,78,18...|
|kenichan i dived ...|     0.0|[kenichan, i, div...|[kenichan, dived,...|[kenichan, dive, ...|(1000,[1,10,225,3...|(1000,[1,10,225,3...|
|my whole body fee...|     0.0|[my, whole, body,...|[whole, body, fee...|[whole, bodi, fee...|(1000,[6,21,335,6...|(1000,[6,21,335,6...|
|nationwideclass n...|     0.0|[nationwid

In [0]:
###### Olası sıfır uzunluklu vektörleri kontrol edin ve kaldırın
@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

print("Total n. of zero-length vectors: {:d}".
      format(features.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors: 39873


In [0]:
# Sıfır uzunluklu vektörlerin var olup olmadığının kontrolü 
features = features.where(num_nonzeros("features") > 0)

print("Total n. of zero-length vectors (after removal): {:d}".
      format(features.where(num_nonzeros("features") == 0).count()))

Total n. of zero-length vectors (after removal): 0


In [0]:
# Veri kümemizi eğitim ve test kümesi olarak ayırma. 0.80 train, 0.20 test
train_df, test_df = features.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

print("Training set size: {:d} instances".format(train_df.count()))
print("Test set size: {:d} instances".format(test_df.count()))

Training set size: 1218676 instances
Test set size: 304387 instances


In [0]:
#logistic regresyon modelini train veri kümesi ile eğitme işlemi
lr = LogisticRegression(labelCol="polarite", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(train_df)
print ("Training is done!")
#Eğitim tamamlandıktan sonra test veri kümesini modele tahmin etmesi için veriyoruz.
prediction = model.transform(test_df)

Training is done!


In [0]:
#Tahmin edilen test veri kümesinden tweet,prediction,polarite değerlerini predictionFinal dataframe'nie atıyoruz.
predictionFinal = prediction.select(
    "tweet", "prediction", "polarite")
#PredictionFinal dataframe değişkenin ilk 4 değerini gösteriyoruz.
predictionFinal.show(n=4, truncate = False)
#Doğru tahmin edilen değerlerinin sayısını gösteriyoruz.
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['polarite']).count()
#Toplam tahmin edilen değerlerin değerini buluyoruz count ile
totalData = predictionFinal.count()
# Doğru tahmin edilen değerlerin sayısını ve modellin accuracy değerini yazdırıyoruz.
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+------------------------------------------------------------------------------------------------------------------------------+----------+--------+
|tweet                                                                                                                         |prediction|polarite|
+------------------------------------------------------------------------------------------------------------------------------+----------+--------+
|a bad nite for the favorite teams astros and spartans lose the nite out with tw was good                                      |0.0       |0.0     |
|a beautiful day and i have to work but hopefully ill get out early to enjoy the sun                                           |4.0       |0.0     |
|a beautiful day even if i have to do more tax work embrace that which you dont want do enjoy what you really want to note self|0.0       |0.0     |
|a beautiful mind is so tragic                                                                            

In [0]:
#y_pred'e tahmin edilen değerleri atıyoruz.
y_pred = prediction.select('prediction').collect()

#yalnızca prediction değerlerini seç ve sütunları etiketle
preds_and_labels = prediction.select(['prediction','polarite'])

#Değerlendirme metirklerini hesaplamak için 
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

precision_0 = metrics.precision(0.0)
recall_0 = metrics.recall(0.0)
precision_4 = metrics.precision(4.0)
recall_4 = metrics.recall(4.0)
f1Score = metrics.fMeasure(0.0,4.0)
print("Summary Stats")
print("Precision 0 = %s" % precision_0)
print("Recall 0 = %s" % recall_0)
print("Precision 4 = %s" % precision_4)
print("Recall 4 = %s" % recall_4)
print("F1 Score = %s" % f1Score)
print(metrics.confusionMatrix().toArray())

Summary Stats
Precision 0 = 0.7753270976196585
Recall 0 = 0.7280611514070293
Precision 4 = 0.7392297894657975
Recall 4 = 0.7851274195794458
F1 Score = 0.730681400169665
[[111820.  41766.]
 [ 32403. 118398.]]


In [0]:
lr = LogisticRegression(labelCol="polarite", featuresCol="features", 
                        maxIter=10)
param_grid = ParamGridBuilder()\
            .addGrid(lr.regParam, [0.0, 0.05, 0.1]) \
            .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
            .build()
dtevaluator = MulticlassClassificationEvaluator(labelCol="polarite", predictionCol="prediction")

cross_val = CrossValidator(estimator=lr, 
                           estimatorParamMaps=param_grid,
                           evaluator = dtevaluator,
                           numFolds=3, # bu değeri büyük seçme 1 olabilir.
                           collectSubModels=True # k-katlama çapraz doğrulaması sırasında eğitilen TÜM modelleri saklamamıza izin verir.
                           )

# Çapraz doğrulamayı çalıştırın ve en iyi parametre grubunu seçin.
cv_model = cross_val.fit(train_df)

/databricks/spark/python/pyspark/ml/util.py:839: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
  warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [0]:
for i, acc in enumerate(cv_model.avgMetrics):
    print("Avg. Accuracy computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, acc))
#En iyi modeli belirleme
print(cv_model.bestModel)

prediction_cross = cv_model.transform(test_df)


Avg. Accuracy computed across k-fold cross validation for model setting #1: 0.756
Avg. Accuracy computed across k-fold cross validation for model setting #2: 0.756
Avg. Accuracy computed across k-fold cross validation for model setting #3: 0.756
Avg. Accuracy computed across k-fold cross validation for model setting #4: 0.756
Avg. Accuracy computed across k-fold cross validation for model setting #5: 0.641
Avg. Accuracy computed across k-fold cross validation for model setting #6: 0.455
Avg. Accuracy computed across k-fold cross validation for model setting #7: 0.756
Avg. Accuracy computed across k-fold cross validation for model setting #8: 0.455
Avg. Accuracy computed across k-fold cross validation for model setting #9: 0.337
LogisticRegressionModel: uid=LogisticRegression_8954d82aa213, numClasses=5, numFeatures=1000


In [0]:
#CrossValidation modelimiz ile tahminleme yapıyoruz.
prediction_cross = cv_model.transform(test_df)

In [0]:
predictionFinal_cross = prediction_cross.select(
    "tweet", "prediction", "polarite")
predictionFinal_cross.show(n=4, truncate = False)
correctPrediction_cross = predictionFinal_cross.filter(
    predictionFinal_cross['prediction'] == predictionFinal_cross['polarite']).count()
totalData_corss = predictionFinal_cross.count()
print("correct prediction:", correctPrediction_cross, ", total data:", totalData_corss, 
      ", accuracy:", correctPrediction_cross/totalData_corss)

+------------------------------------------------------------------------------------------------------------------------------+----------+--------+
|tweet                                                                                                                         |prediction|polarite|
+------------------------------------------------------------------------------------------------------------------------------+----------+--------+
|a bad nite for the favorite teams astros and spartans lose the nite out with tw was good                                      |0.0       |0.0     |
|a beautiful day and i have to work but hopefully ill get out early to enjoy the sun                                           |4.0       |0.0     |
|a beautiful day even if i have to do more tax work embrace that which you dont want do enjoy what you really want to note self|0.0       |0.0     |
|a beautiful mind is so tragic                                                                            